<a href="https://colab.research.google.com/github/NainaMKumar/Myocardial_Infarction_Detection/blob/main/MYOCARDIAL_INFARCTION_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from pathlib import Path
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam
from keras.metrics import Precision
from keras.metrics import Recall
from keras.metrics import AUC
from keras.utils import to_categorical

In [ ]:
data = Path('/content/Heart Attack Dataset.zip')

In [ ]:
from zipfile import ZipFile

with ZipFile(data, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
data = pd.read_csv('/content/Heart Attack Dataset/Medicaldataset.csv')

In [ ]:
display(data)

,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result
0,64,1,66,160,83,160.0,1.80,0.012,negative
1,21,1,94,98,46,296.0,6.75,1.060,positive
2,55,1,64,160,77,270.0,1.99,0.003,negative
3,64,1,70,120,55,270.0,13.87,0.122,positive
4,55,1,64,112,65,300.0,1.08,0.003,negative
...,...,...,...,...,...,...,...,...,...
1314,44,1,94,122,67,204.0,1.63,0.006,negative
1315,66,1,84,125,55,149.0,1.33,0.172,positive
1316,45,1,85,168,104,96.0,1.24,4.250,positive
1317,54,1,58,117,68,443.0,5.80,0.359,positive


## **EDA** 📊

In [ ]:
ages_distribution = px.histogram(data, x = 'Age')
ages_distribution.show()

In [ ]:
gender_counts = data['Gender'].value_counts()
print(gender_counts)

gender_count = px.bar(gender_counts, title = "Gender Distribution")
gender_count.show()

1    870
0    449
Name: Gender, dtype: int64


In [ ]:
result_counts = data['Result'].value_counts()

result_count = px.bar(result_counts, title = "Positive vs. Negative Counts")
result_count.show()

In [ ]:
X = data.iloc[:, data.columns != 'Result']
y = data.iloc[:, data.columns == 'Result']

In [ ]:
#figure out which features are useful

estimator = LogisticRegression()
rfe = RFE(estimator, n_features_to_select=10)
rfe.fit(X, y)

selected_features = rfe.support_
print(selected_features)

feature_rankings = rfe.ranking_
print(feature_rankings)


[ True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
X = np.array(X)

In [ ]:
y = y.replace({'positive': 1, 'negative': 0})

display(y)

,Result
0,0
1,1
2,0
3,1
4,0
...,...
1314,0
1315,1
1316,1
1317,1


In [ ]:
num_classes = len(np.unique(y))

#class weights
total_samples = len(y)
positive_samples = y.sum()
negative_samples = total_samples - positive_samples

class_weight = {0: total_samples / negative_samples, 1: total_samples / positive_samples}
print(class_weight)

{0: Result    2.591356
dtype: float64, 1: Result    1.628395
dtype: float64}


In [ ]:
y = np.array(y)
y = y.ravel()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
lgb_classifier = lgb.LGBMClassifier(max_depth = 2, learning_rate = 0.001, class_weight = class_weight,
                                    reg_alpha = 0.5, reg_lambda = 0.5)
lgb_classifier.fit(X_train, y_train)

y_pred = lgb_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       155
           1       0.98      0.98      0.98       241

    accuracy                           0.98       396
   macro avg       0.98      0.98      0.98       396
weighted avg       0.98      0.98      0.98       396

